In [1]:
# Remember you need to use SWAN with NXCALS support....
from datetime import datetime
from datetime import timedelta
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector as msql
import pytimber
import os
ldb = pytimber.LoggingDB(source="nxcals")
#this is the new pytimber access, not working yet
#ldb = pytimber.LoggingDB(spark_session = spark)
#ldb = pytimber.LoggingDB(spark_session = spark, sparkconf="large",
#    sparkprops={
#        "spark.driver.cores", "4",
#        "spark.executor.memory", "10g",
#        "spark.sql.parquet.columnarReaderBatchSize", "32",
#        "spark.driver.maxResultSize", "1t",
#        "spark.task.maxDirectResultSize", "1t"
#})

kinit: type the password for `regenfus`:········
Password for regenfus@CERN.CH: 


In [2]:
# pack daily data into text file     
# pickup signals       
BmExtrALPH  = 'LNE.APULB.0030:Acquisition:totalIntensitySingle' # Extr into 00 beam line (alpha,aegis,base ...
BmExtrGBAR  = 'LNE.APULB.5030:Acquisition:totalIntensitySingle' # Extr into 50 beam line (gbar, puma ...
SEM0008in   = 'LNE00.BSGW.0008:Acquisition:isMonitorIn' # SEM 0008 in
SEM0015in   = 'LNE00.BSGW.0015:Acquisition:isMonitorIn' # SEM 0015 in
SEM0025in   = 'LNE00.BSGW.0025:Acquisition:isMonitorIn' # SEM 0025 in
SEM5020in   = 'LNE50.BSGW.5020:Acquisition:isMonitorIn' # SEM 5020 in
SEM5060in   = 'LNE50.BSGW.5060:Acquisition:isMonitorIn' # SEM 5060 in
BmInjAD     = 'LNR.APULB.0227:Acquisition:totalIntensitySingle' # Inj into AD
CycleName   = 'ELENA.LSA:CYCLE'                         # Cycle name
 
print('Starting')

yr = 23
mt = 11
for dy in range(2,3):
    
    date = "20%02d-%02d-%02d " % (yr,mt,dy)    
    fila = "ELENA_BM_%02d_%02d_%02d.csv" % (yr,mt,dy)
    filb = "ELENA_Cycles_%02d_%02d_%02d.csv" % (yr,mt,dy)
    fold = "/eos/experiment/gbar/elena/data/%02d_%02d_%02d" % (yr,mt,dy)
    if not os.path.exists(fold):
       os.makedirs(fold)
    fdata = fold + "/" + fila
    fcycl = fold + "/" + filb
    
    t1 = date + '00:00:00.000'
    t2 = date + '23:59:59.999'
    
    dat  = ldb.get([BmExtrALPH,BmExtrGBAR,SEM0008in,SEM0015in,SEM0025in,SEM5020in,SEM5060in,BmInjAD],t1,t2)
    kys  = list(dat)
    
    cyc  = ldb.get([CycleName],t1,t2)
    kyc  = list(cyc)
    # print(kyc)
    # print(cyc[kyc[0]][0][-1])

    evs  = len(dat[kys[0]][0])
    evc  = len(cyc[kyc[0]][0])
    print(date,' : found ',evs,' and ',evc,' entries')
    
    # write a comma separated data file 
    fil = open(fdata, "w")
    # header
    fil.write("%d entries: Unix_Time,%s,%s,%s,%s,%s,%s,%s,%s\n" % (evs,BmExtrALPH,BmExtrGBAR,
                                       SEM0008in,SEM0015in,SEM0025in,SEM5020in,SEM5060in,BmInjAD))            
    for j in range(evs):
        # first is timestamp from extr to GBAR
        fil.write("%.3f,%.3f,%.3f," % (dat[kys[1]][0][j],dat[kys[0]][1][j],dat[kys[1]][1][j]))
        fil.write("%d,%d,%d,"       % (dat[kys[2]][1][j],dat[kys[3]][1][j],dat[kys[4]][1][j]))
        fil.write("%d,%d,%.3f\n"    % (dat[kys[5]][1][j],dat[kys[6]][1][j],dat[kys[7]][1][j]))        
    fil.close()    
    
    # write a comma separated file with time stamps and the Elena cycles 
    fil = open(fcycl, "w")
    # header
    fil.write("%d entries: Unix_Time,%s\n" % (evc,CycleName))            
    for j in range(evc):
        if cyc[kyc[0]][1][j] != 'ELENA_ZERO_CYCLE' :
            fil.write("%.3f,%s\n" % (cyc[kyc[0]][0][j],cyc[kyc[0]][1][j]))        
    fil.close()
    
print("finished")

# Out: 
# Starting
# 2023-08-31   : found  1689  and  51306  entries
# finished

Starting
2023-11-02   : found  2140  and  45806  entries
finished


In [ ]:
# test stuff # test stuff # test stuff # test stuff
Starting
2023-10-01   : found  733  entries
2023-10-02   : found  1414  entries
2023-10-03   : found  2006  entries
2023-10-04   : found  5247  entries
2023-10-05   : found  5135  entries
2023-10-06   : found  4067  entries
2023-10-07   : found  747  entries
2023-10-08   : found  707  entries
2023-10-09   : found  1472  entries
2023-10-10   : found  1324  entries
finished

Starting
2023-10-15   : found  711  entries
2023-10-16   : found  1692  entries
2023-10-17   : found  2205  entries
2023-10-18   : found  2030  entries
2023-10-19   : found  1195  entries
finished

yr = 23
mt = 10
dy = 9

#dt = '20' + str(yr) + '-' + str(mt) + '-' + str(dy) + ' '            # data aquire limits 
#fd = 'ELENA_BM_' + str(yr) + '_' + str(mt) + '_' + str(dy) + '.csv'  # file name

# format
#num = 3
#print("%02d_%02d_%02d" % (num,num,num))
# basically % is like printf or sprintf
#print(format(num,'02d'))  # old

dt = "20%02d-%02d-%02d " % (yr,mt,dy)
fd = "ELENA_BM_%02d_%02d_%02d.csv" % (yr,mt,dy)
t1 = dt + '00:00:00.000'
t2 = dt + '23:59:59.999'

print(dt)
print(fd)
print(t1)
print(t2)

fold = "/eos/experiment/gbar/elena/data/%02d_%02d_%02d" % (yr,mt,dy)
import os
if not os.path.exists(fold):
   os.makedirs(fold)
ff = fold + "/" + fd 
print(ff)

evs = 500
SEM0008in   = 'LNE00.BSGW.0008:Acquisition:isMonitorIn' # SEM 0008 in
SEM0015in   = 'LNE00.BSGW.0015:Acquisition:isMonitorIn' # SEM 0015 in
SEM0025in   = 'LNE00.BSGW.0025:Acquisition:isMonitorIn' # SEM 0025 in

print("%d entries: Unix_Time,%s,%s,%s\n" % (evs,SEM0008in,SEM0015in,SEM0025in))
    
    
#'UNIX_TIME,'+BmExtrALPH+','+BmExtrGBAR+','+SEM0008in+','+
# SEM0015in+','+SEM0025in+','+SEM5020in+','+SEM5060in+','+BmInjAD+'\n')

q = 1697642983.733
r = 1697643016.133
s = 49.354
w = 0

print("%.3f,%.3f,%.3f,%d,%d,%d,%d,%d,%.3f\n" % (dat[kys[1]][0][j],dat[kys[]][1][j]))

#    for j in range(evs):
        # timestamp from extr to GBAR
#        fil.write(str(dat[kys[1]][0][j]))
        # data
#        for k in range(8):
#            fil.write(','+str(dat[kys[k]][1][j]))
#        fil.write('\n')    

#for i in range(8):
#    print(i,kys[i])
# check 
# 0 LNE.APULB.0030:Acquisition:totalIntensitySingle
# 1 LNE.APULB.5030:Acquisition:totalIntensitySingle
# 2 LNE00.BSGW.0008:Acquisition:isMonitorIn
# 3 LNE00.BSGW.0015:Acquisition:isMonitorIn
# 4 LNE00.BSGW.0025:Acquisition:isMonitorIn
# 5 LNE50.BSGW.5020:Acquisition:isMonitorIn
# 6 LNE50.BSGW.5060:Acquisition:isMonitorIn
# 7 LNR.APULB.0227:Acquisition:totalIntensitySingle

#Timestamp (UNIX_TIME),  
#for i in range(8):     # test-print time stamps of last event
#    print(dat[kys[i]][0][-1])

print(type(InjInt))
plt.plot(InjInt['LNE.APULB.0227:INTENSITY'])
#plt.plot(AD_intensity['DR.RF.NB-PART.100M_E:INTENSITY'][1])
plt.xlabel('shot number')
plt.ylabel('charge e+6')

# Intensity
plt.figure()
plt.plot(InjInt['LNR.APULB.0227:Acquisition:totalIntensitySingle'], label='LNR.APULB.0227')
#plt.plot(InjInt['LNE50.BSGW.5020:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5060 H')
#plt.xlabel("shot number")
#plt.ylabel("Intensity [a.u.]")
plt.ylim([0, 2.5e3])
plt.legend()

# Intensity
plt.figure()
plt.plot(SEM_intensity['LNE50.BSGW.5020:FitAcq:gaussAmplitude'][1][:,0], label='LNE50.5020 V')
plt.plot(SEM_intensity['LNE50.BSGW.5020:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_intensity['LNE50.BSGW.5060:FitAcq:gaussAmplitude'][1][:,1], label='LNE50.5060 H')
plt.xlabel("shot number")
plt.ylabel("Intensity [a.u.]")
plt.ylim([0, 2.5e3])
plt.legend()

# Position
plt.figure()
plt.plot(SEM_position['LNE50.BSGW.5020:FitAcq:gaussMean'][1][:,0], label='LNE50.5020 V')
plt.plot(SEM_position['LNE50.BSGW.5020:FitAcq:gaussMean'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_position['LNE50.BSGW.5060:FitAcq:gaussMean'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_position['LNE50.BSGW.5060:FitAcq:gaussMean'][1][:,1], label='LNE50.5060 H')
plt.xlabel("shot number")
plt.ylabel("<position> [mm]")
plt.ylim([-10, 10])
plt.legend()

# Sigma
plt.figure()
plt.plot(SEM_sigmas['LNE50.BSGW.5020:FitAcq:gaussSigma'][1][:,0], label='LNE50.5020 V')
plt.plot(SEM_sigmas['LNE50.BSGW.5020:FitAcq:gaussSigma'][1][:,1], label='LNE50.5020 H')
#plt.plot(SEM_sigmas['LNE50.BSGW.5060:FitAcq:gaussSigma'][1][:,0], label='LNE50.5060 V')
#plt.plot(SEM_sigmas['LNE50.BSGW.5060:FitAcq:gaussSigma'][1][:,1], label='LNE50.5060 H')
plt.xlabel("shot number")
plt.ylabel("\sigma [mm]")
plt.ylim([0, 2.5])
plt.legend()


#Extraction Intensity
plt.figure()
plt.plot(EXT_intensity['LNE.APULB.5030:INTENSITY'][0], label='LNE.5030 V')
#plt.plot(EXT_intensity['LNE.APULB.5030:INTENSITY'][1], label='LNE.5030 H')
plt.xlabel('shot number')
plt.ylabel('Charges E6')
plt.legend()